<a href="https://colab.research.google.com/github/Rt247/Not_NLP_CW/blob/BERT_method/NLP_CW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import urllib
import json

url_hash = "384830ba34a2c9a1dfeb869062d8c6dc/raw/729c688d2bb4801a135a1f3824f30311a84de788"

data_set_load = {
  "url_dev_enzh_mt": f'https://gist.githubusercontent.com/cheukwing/{url_hash}/dev.enzh.mt',
  "url_dev_enzh_scores": f'https://gist.githubusercontent.com/cheukwing/{url_hash}/dev.enzh.scores',
  "url_dev_enzh_src": f'https://gist.githubusercontent.com/cheukwing/{url_hash}/dev.enzh.src',
  "url_train_enzh_mt": f'https://gist.githubusercontent.com/cheukwing/{url_hash}/train.enzh.mt',
  "url_train_enzh_scores": f'https://gist.githubusercontent.com/cheukwing/{url_hash}/train.enzh.scores',
  "url_train_enzh_src": f'https://gist.githubusercontent.com/cheukwing/{url_hash}/train.enzh.src',
  "url_test_enzh_src": f'https://gist.githubusercontent.com/cheukwing/{url_hash}/test.enzh.src',
  "url_test_enzh_mt": f'https://gist.githubusercontent.com/cheukwing/{url_hash}/test.enzh.mt',
}

data_sets = {}
for key, value in data_set_load.items():
  r = urllib.request.urlopen(value)
  data = r.read().splitlines()
  data_sets[key[4:]] = data
  print(data_sets[key[4:]][0])

print(data_sets.keys())

In [0]:
!pip install transformers
import torch
from transformers import BertTokenizer, BertModel

In [0]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# Tokenize our sentence with the BERT tokenizer.
original_texts = [text.decode() for text in data_sets['train_enzh_src']]
translated_texts = [text.decode('utf8') for text in data_sets['train_enzh_mt']]

text_pairs = list(zip(original_texts, translated_texts))


inputs = [tokenizer.encode_plus(original, text_pair=translated, add_special_tokens = True, pad_to_max_length=True) for original, translated in text_pairs]

input_ids = [d['input_ids'] for d in inputs]
input_segments = [d['token_type_ids'] for d in inputs]
input_attention_masks = [d['attention_mask'] for d in inputs]

# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor(input_ids)
segments_tensor = torch.tensor(input_segments)
attention_masks_tensor = torch.tensor(input_attention_masks)

# # Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-multilingual-cased')

# # Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

In [0]:
# # Predict hidden states features for each layer
# with torch.no_grad():
#     last_hidden_states = model(tokens_tensor, segments_tensors)

# features = last_hidden_states[0][:][0][:].numpy()
# print(features)